In [1]:
from kbmod.work_unit import WorkUnit
from astropy.table import Table
from kbmod.run_search import SearchRunner
import numpy as np

In [2]:
t = Table.read("inframe_fakes_b1h.ecsv")

## Setup

before we run a targeted search for a specific object, we need to pick an object to test on!

In [3]:
orbids = np.unique(t["ORBITID"])
orbids

204287
489097
587985
601439
1533755
1668438
2674005
2706546
2820143
3581634
3590834


These are our unique orbit ids. To select the rows with all the possible, let's select a specific orbit id as `oid` and get all the positions associated with that object.

In [4]:
oid = orbids[2]
t0 = t[t["ORBITID"] == oid]
t0

RA,DEC,EXPNUM,CCDNUM,ORBITID,aei,mjd_mid,TDB,xv,H_VR,AMP,PERIOD,PHASE,d,observatory,r,MAG,type
float64,float64,int32,int64,int64,float64[6],float64,float64,float64[6],float64,float64,float64,float64,float64,float64[3],float64,float64,str3
355.9623580564759,-4.692830042823854,1031142,47,587985,58.9031828419715 .. 80.20324305172379,59466.07420776917,21.688087290660633,51.590791778842664 .. 0.5924326416950347,8.390057581774474,0.0,0.0,0.0,51.904578423449564,0.9706285011680602 .. -0.09292801567473531,50.91672885048846,25.5003883946863,tno
355.9623311602185,-4.692840279898922,1031143,47,587985,58.9031828419715 .. 80.20324305172379,59466.07594147999,21.688092037302113,51.59079143828357 .. 0.5924326474087211,8.390057581774474,0.0,0.0,0.0,51.904577610342955,0.9706353935996239 .. -0.09291653346158245,50.916724008539624,25.500388154172153,tno
355.9623043755052,-4.692850472750195,1031144,47,587985,58.9031828419715 .. 80.20324305172379,59466.077667741774,21.68809676354931,51.590791099187385 .. 0.5924326530978548,8.390057581774474,0.0,0.0,0.0,51.90457680072995,0.9706422531069862 .. -0.0929051004968976,50.91671919097193,25.500387914843937,tno
355.96227771447303,-4.692860616792844,1031145,47,587985,58.9031828419715 .. 80.20324305172379,59466.07938577787,21.68810146727583,51.59079076170667 .. 0.592432658759876,8.390057581774474,0.0,0.0,0.0,51.90457599497479,0.9706490766098764 .. -0.09289372192598723,50.916714399941156,25.50038767680883,tno
355.9622509061528,-4.692870815133429,1031146,47,587985,58.9031828419715 .. 80.20324305172379,59466.08111305117,21.688106196292438,51.590790422411146 .. 0.5924326644523363,8.390057581774474,0.0,0.0,0.0,51.904575184887435,0.9706559334251368 .. -0.09288228208954079,50.91670958678809,25.500387437649007,tno
355.9622239878634,-4.692881053560917,1031147,47,587985,58.9031828419715 .. 80.20324305172379,59466.08284715607,21.688110944012806,51.59079008177333 .. 0.5924326701673074,8.390057581774474,0.0,0.0,0.0,51.90457437159612,0.9706628139171125 .. -0.09287079691877195,50.9167047583036,25.50038719770127,tno
355.96219741057575,-4.692891160579582,1031148,47,587985,58.9031828419715 .. 80.20324305172379,59466.0845590463,21.688115630912762,51.59078974549891 .. 0.5924326758090637,8.390057581774474,0.0,0.0,0.0,51.90457356872345,0.9706696028442408 .. -0.09285945879333177,50.916699995352026,25.500386960984205,tno
355.96217065504476,-4.692901333679148,1031149,47,587985,58.9031828419715 .. 80.20324305172379,59466.08628217192,21.688120348573648,51.59078940701715 .. 0.5924326814878443,8.390057581774474,0.0,0.0,0.0,51.90457276058142,0.9706764328574085 .. -0.09284804616712972,50.916695204865455,25.500386722872374,tno
355.9621440489389,-4.692911448280507,1031150,47,587985,58.9031828419715 .. 80.20324305172379,59466.08799543176,21.68812503922345,51.59078907047305 .. 0.5924326871341076,8.390057581774474,0.0,0.0,0.0,51.90457195706646,0.9706832202768626 .. -0.09283669879623334,50.916690445546315,25.500386486283315,tno


Now we can load in our `WorkUnit` for the b1h field and convert the longitude and latitude coordinates into pixel coordinates.

In [5]:
wu = WorkUnit.from_fits("/epyc/projects/kbmod/data/20210908_B1h_047_test_data/20210908_B1h_047.wu")

Loading images: 100%|██████████| 70/70 [00:02]


In [6]:
xs, ys = wu.get_pixel_coordinates(t0["RA"], t0["DEC"])

Now we have a list of x and y coordinates, `xs` and `ys`. Before we go any further, we should confirm that given orbit is going to be roughly linear in pixel trajectory terms. Basically we want to make sure that the observations are lined up in such a way that `KBMOD` will actually find the object while searching the image. to do this, we need to calculate the changes in pixels in the x and y direction and compare the mean, maximum and minimum.

In [7]:
dx = []
for i in range(len(xs) - 2):
    dx.append(xs[i+1] - xs[i])

dy = []
for i in range(len(ys) - 2):
    dy.append(ys[i+1] - ys[i])

Generally, the min and max for the `dx` and `dy` shouldn't be much greater than +-1. If it is more than that, this orbit id might not be a good choice for actually searching for in `KBMOD`.

In [8]:
print(np.mean(dx), np.min(dx), np.max(dx))
print(np.min(dx) - np.mean(dx), np.max(dx) - np.mean(dx))

0.36572154036143883 0.36191562580597747 0.3714552552683017
-0.003805914555461365 0.0057337149068628634


In [9]:
print(np.mean(dy), np.min(dy), np.max(dy))
print(np.min(dy) - np.mean(dy), np.max(dy) - np.mean(dy))

-0.1391383572958685 -0.14084709058352018 -0.1380503450350261
-0.001708733287651698 0.0010880122608423748


now we have to calculate the pixels per day velocity for both the x and y. To do this we'll take the difference between the first and last x and y positions and divide the total time elapsed between the start and end of the search.

In [10]:
time_diff = t0["mjd_mid"][-1] - t0["mjd_mid"][0]

In [11]:
mean_vx = (xs[-1] - xs[0]) / time_diff
mean_vx

212.20471144171128

In [12]:
mean_vy = (ys[-1] - ys[0]) / time_diff
mean_vy

-80.72805043936555

Now we can use the `mean_vx` and `mean_vy` as a starting point to try and guess where the velocity is and where `KBMOD` will find the object.

I put in a simple search that worked to find `orbids[0]` here but feel free to play around with this generator config. The step values is the difference in velocities that `KBMOD` will search around.

In [13]:
gen = {
    "name": "VelocityGridSearch",
    "vx_steps": 25,
    "min_vx": mean_vx - .125,
    "max_vx": mean_vx + .125,
    "vy_steps": 25,
    "min_vy": mean_vy - .125,
    "max_vy": mean_vy + .125,
}

In [14]:
wu.config.set("generator_config", gen)
wu.config.set("do_clustering", False)
wu.config.set("sigmaG_filter", False)
# wu.config.set("peak_offset_max", 6)
# wu.config.set("predictive_line_cluster", True)
wu.config.set("lh_level", 10.0)

In [15]:
wu.config.set("result_filename", f"./{oid}_pencil.ecsv")

In [16]:
wu.reprojection_frame = "original"

In [17]:
r = SearchRunner().run_search_from_work_unit(wu)

Configuration file ./587985_pencil_provenance/587985_pencil_config.yaml already exists.


if you need to quit out of the kernel and reload with the saved result file feel free to comment this out.

In [18]:
# from kbmod.results import Results
# r = Results.read_table(f"{oid}_pencil.ecsv")

In [19]:
r

x,y,vx,vy,likelihood,flux,obs_count,uuid,psi_curve,phi_curve,coadd_mean,coadd_sum,coadd_weighted,coadd_median,stamp,pred_x,pred_y,global_ra,global_dec,img_ra,img_dec,img_x,img_y
int64,int64,float64,float64,float64,float64,int64,str32,float32[70],float32[70],"float32[51,51]","float32[51,51]","float32[51,51]","float32[51,51]","float32[51,51]",float64[70],float64[70],float64[70],float64[70],float64[70],float64[70],float64[70],float64[70]
1559,1458,212.1213836669922,-80.83221435546875,287.7248229980469,1696.322998046875,70,7101d8750c1a49e6adde5c4af8019118,0.5689805 .. 0.7691034,0.00029850876 .. 0.00046166545,-0.17986986 .. -0.69165635,-8.633754 .. -48.415943,-0.19618714 .. -0.6102753,-0.8095312 .. -0.74952185,-8.633754 .. -48.415943,1559.5 .. 1584.7560280551345,1458.5 .. 1448.8757903223689,356.12291811061004 .. 356.1210656035812,-4.593839470680535 .. -4.594543254115002,356.12291811061004 .. 356.1210656035812,-4.593839470680535 .. -4.594543254115002,652.7504015385015 .. 674.6073257057955,2616.3949205113017 .. 2585.122342941855
1560,1458,212.1526336669922,-80.8530502319336,282.9046936035156,1668.1834716796875,70,9587a5d5d8e443d4a58334fa3b2f5ed6,0.61465746 .. 0.746981,0.00029922614 .. 0.0004624948,-0.69428194 .. -1.0929357,-33.325535 .. -76.5055,-0.7046677 .. -0.9635007,-1.4597648 .. -0.6271755,-33.325535 .. -76.5055,1560.5 .. 1585.7597488062586,1458.5 .. 1448.873309518824,356.1228447598907 .. 356.12099197987595,-4.593839475066294 .. -4.594543439711856,356.1228447598907 .. 356.12099197987595,-4.593839475066294 .. -4.594543439711856,652.7533747693527 .. 674.6127667089199,2615.3963502900638 .. 2584.120045332067
1559,1460,212.1526336669922,-80.8530502319336,175.24517822265625,1032.6661376953125,70,2d1f2bda905640fe96c5d35df3211d10,0.26220676 .. 0.6569101,0.00029909646 .. 0.00046219045,-0.14952278 .. -0.20116161,-6.728525 .. -14.081312,0.038526207 .. -0.2052932,-0.96490186 .. -0.27401498,-6.728525 .. -14.081312,1559.5 .. 1584.7597488062586,1460.5 .. 1450.873309518824,356.12291810174077 .. 356.12106532218496,-4.593693240503761 .. -4.594397205332561,356.12291810174077 .. 356.12106532218496,-4.593693240503761 .. -4.594397205332561,650.7509169522435 .. 672.6099762364946,2616.3905098472014 .. 2585.114196211089
1560,1460,212.1317901611328,-80.8530502319336,173.31353759765625,1021.5646362304688,70,846e27b7f1d441898b9eae0902b522e6,0.2832139 .. 0.641127,0.000297896 .. 0.0004629433,-0.32746512 .. -0.25930476,-14.73593 .. -18.151335,-0.13370207 .. -0.25082272,-1.3548137 .. -0.42041475,-14.73593 .. -18.151335,1560.5 .. 1585.7572670943273,1460.5 .. 1450.873309518824,356.12284475103655 .. 356.1209921534399,-4.593693244889391 .. -4.594397209519069,356.12284475103655 .. 356.1209921534399,-4.593693244889391 .. -4.594397209519069,650.7538901428745 .. 672.6129182707554,2615.391939958409 .. 2584.11808704898
639,616,212.1526336669922,-80.8530502319336,164.85308837890625,967.556396484375,70,6ea291189a04442f88e6d279919a1317,0.113291815 .. 0.22214693,0.00029753157 .. 0.00046073447,1.1159198 .. -0.39984915,78.11439 .. -27.58959,1.0556415 .. -0.5393471,0.6724614 .. -0.18380404,78.11439 .. -27.58959,639.5 .. 664.7597488062586,616.5 .. 606.873309518824,356.19041028342934 .. 356.1885574152595,-4.655395080870874 .. -4.656099222678229,356.19041028342934 .. 356.1885574152595,-4.655395080870874 .. -4.656099222678229,1491.9210939599106 .. 1513.8338468333352,3537.1874780923945 .. 3505.8620897219726
640,616,212.1109619140625,-80.8530502319336,161.81143188476562,949.7457885742188,70,985668a772a94855a72a810a39ac7b77,0.08801198 .. 0.2758973,0.00029628177 .. 0.00046429777,2.070465 .. -0.90141857,144.93256 .. -62.197884,1.9336678 .. -1.1062787,2.869084 .. -1.7840825,144.93256 .. -62.197884,640.5 .. 665.7547871991688,616.5 .. 606.873309518824,356.1903369265634 .. 356.18848442227994,-4.65539509230552 .. -4.6560992338669,356.1903369265634 .. 356.18848442227994,-4.65539509230552 .. -4.6560992338669,1491.9238489478055 .. 1513.8365876357664,3536.1889390074543 .. 3504.86832164353
744,6

If the following check doesn't result in any rows than it's likely that the search didn't find the object in question. You can still check around in the above table to see if there's something close to the original object in starting x,y and vx,vy but this is probably the fastest way to check if the object was recovered!

In [20]:
r[(r["x"] >= xs[0] - 1) & (r["x"] <= xs[0] + 1) & (r["y"] >= ys[0] - 1) & (r["y"] <= ys[0] + 1)]

x,y,vx,vy,likelihood,flux,obs_count,uuid,psi_curve,phi_curve,coadd_mean,coadd_sum,coadd_weighted,coadd_median,stamp,pred_x,pred_y,global_ra,global_dec,img_ra,img_dec,img_x,img_y
int64,int64,float64,float64,float64,float64,int64,str32,float32[70],float32[70],"float32[51,51]","float32[51,51]","float32[51,51]","float32[51,51]","float32[51,51]",float64[70],float64[70],float64[70],float64[70],float64[70],float64[70],float64[70],float64[70]
